<a href="https://colab.research.google.com/github/ghaafs/Dynamic-Scheduling/blob/main/Tesla_Stock_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'tesla-stock-price-with-indicators-10-years:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4582525%2F7821310%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240316%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240316T141949Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D61ace5e26e8af5f852bb23e562a946c22b26f990ce466947b99550202af53ed777f989ccfc3cb4e4177f8123cbf4490c81c174581d86ea7b555330318a72c4c225193654c8b074fe185ff42909dab81861bc392329a612f05fb29be8b55bd78d7845e6a2607869dd56e108e83d1ce1b28fc527c2090671325f1fd76aeeef16d8142e9c065a4ae4355b71682b9eaac7ab4db136cafd41921a475bb428fa936a21414190f9b17b1474fc13dc6c56347046453840cc5f5f6b9ef04bfe54072db3395daf208de8bee8ff446d063fb8cb1b3c7294d119273be6b3882b2472f882b45f6510a506fa1f9bd0a94df45a1e807a52383fc2ee3926ce30968e7bd4995839e6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.colors
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from datetime import datetime, timedelta
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# Read Source Data

In [ ]:
df=pd.read_csv("/kaggle/input/tesla-stock-price-with-indicators-10-years/tsla_2014_2023.csv")
df['date']=pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df=df.set_index('date')
df.head()

### Plot Close Price

In [ ]:
plt.figure(figsize=(15, 6))
plt.ylabel('Close Price')
plt.xlabel('Date')
plt.title(f"TSLA: Close Price - 2014 to 2023")
df['close'].plot()
plt.tight_layout()

## Plot Volume

In [ ]:
plt.figure(figsize=(15, 6))
plt.ylabel('Volume')
plt.xlabel('Date')
plt.title(f"TSLA: Volume - 2014 to 2023")
df['volume'].plot()
plt.tight_layout()

## Plot RSI

In [ ]:
plt.figure(figsize=(15, 6))
plt.ylabel('Volume')
plt.xlabel('Date')
plt.title(f"TSLA: RSI - 2014 to 2023")
df['rsi_7'].plot()
df['rsi_14'].plot()
plt.tight_layout()

## Plot SMA and EMA

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1)
fig.set_figheight(6)
fig.set_figwidth(15)
df[['sma_50','sma_100','ema_50','ema_100']].plot(ax=axes)
axes.set_title('TSLA: SMA/EMA - 2014 to 2023')
plt.show()

## Average Returns

In [ ]:
df=df.reset_index()
df['daily_return']=df['next_day_close']-df['close']
df.head()

# Plot Closing Price And Shares Traded

In [ ]:
init_notebook_mode(connected=True)
temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), width=800))
colors=px.colors.qualitative.Plotly
#
df_date=df.date.unique()
close_avg=df.groupby('date')['close'].mean().rename('Closing Price')
vol_avg=df.groupby('date')['volume'].mean().rename('Volume')

fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True)
for i, j in enumerate([close_avg, vol_avg]):
    fig.add_trace(go.Scatter(x=df_date, y=j, mode='lines',
                             name=j.name, marker_color=colors[i]), row=i+1, col=1)
fig.update_xaxes(rangeslider_visible=False,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(count=2, label="2y", step="year", stepmode="backward"),
                         dict(step="all")])),
                 row=1,col=1)
fig.update_layout(template=temp,title='Closing Price, and Shares Traded',
                  hovermode='x unified', height=800,
                  yaxis1_title='Closing Price', yaxis2_title='Shares Traded',
                  showlegend=False)
fig.show()

# Tesla Stock Analysis Chart

In [ ]:
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(df, x=df.date, y=[df['open'], df['close'],
                                          df['high'], df['low']],
             labels={'date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()